In [21]:
!pip install beautifulsoup4
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-text-splitter
!pip install openai
!pip install faiss-cpu

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter


In [39]:
import getpass
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
import warnings
import openai
import langchain_openai
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document

warnings.filterwarnings('ignore')

In [23]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [24]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

6923

In [25]:
print(docs[0].page_content[500:1000])

os do Recife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.[2]
Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões.[1][3][4][5]
O CES


### Dividindo Documentos - Splitting/Chunking

In [26]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

10

In [27]:
print(all_splits[2].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Indexando - Store

In [28]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [29]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Onde está localizado o CESAR?")

len(retrieved_docs)

6

In [30]:
print(retrieved_docs[0].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Buscando e Recuperando Informações - Retrieve

In [31]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [32]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [35]:
for chunk in rag_chain.stream("Onde fica o CESAR?"):
    print(chunk, end="", flush=True)

O CESAR, ou Centro de Estudos e Sistemas Avançados do Recife, fica na cidade do Recife, Pernambuco. Ele também possui filiais em Manaus, Curitiba e Sorocaba.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [40]:
texto = """
O Sonho de Espirrolho e a Semente Encantada

Espirrolho Folhalhudo, um firbolg conhecido na vila por seu nome peculiar e risada contagiante, vivia tranquilamente cuidando das plantas que adornavam os arredores da Floresta Sombria. Ele era grande, com orelhas longas e olhos bondosos, e tinha o dom de ouvir o que as árvores sussurravam ao vento.

Uma noite, Espirrolho teve um sonho estranho. Ele viu um ser majestoso emergir da terra, formado por raízes, pedras e musgo. O ser, com olhos brilhando como vaga-lumes, entregou-lhe algo pequeno e luminoso, sussurrando: "Proteja os segredos da floresta." Ao despertar, Espirrolho encontrou uma semente na palma da mão. Ela pulsava com uma energia mágica, emanando um brilho esverdeado.

Espirrolho sabia que aquilo era mais do que um presente: era uma missão. Ele tentou plantar a semente, mas a terra ao redor parecia rejeitá-la, tremendo levemente como se recusasse recebê-la. O firbolg, então, decidiu consultar a Guardiã Verde, uma dríade sábia que vivia no coração da floresta.

O Caminho para a Guardiã Verde

A jornada foi cheia de desafios. Conforme Espirrolho se aventurava pelas profundezas da floresta, a semente parecia atraí-lo para lugares misteriosos. Ele encontrou clareiras cobertas de flores que nunca tinha visto e animais que o observavam com curiosidade incomum. Contudo, a floresta também começou a mudar. Partes dela murchavam, como se a magia estivesse enfraquecendo.

Finalmente, Espirrolho chegou à Guardiã Verde. Ela olhou para a semente com reverência e preocupação. "Essa é a Semente da Origem," disse ela. "Ela contém a essência da floresta e seus segredos mais antigos. Mas algo está errado. Alguém ou algo a retirou de seu lugar de descanso."

O Confronto com o Usurpador

A Guardiã explicou que o equilíbrio da floresta estava em perigo e que Espirrolho deveria levar a semente ao Monte Raiz Profunda, onde ela deveria ser replantada. No entanto, um usurpador, conhecido como o Feiticeiro Cinzento, estava drenando a magia da floresta para seus próprios propósitos e tentaria impedir Espirrolho.

Espirrolho, com coragem inesperada, enfrentou o Feiticeiro Cinzento em um vale repleto de árvores mortas. A semente em sua mão brilhou intensamente, mostrando-lhe como usar sua magia para convocar a força da floresta. Raízes emergiram do chão, prendendo o feiticeiro, enquanto Espirrolho corria para o Monte Raiz Profunda.

O Renascimento da Floresta

No topo do monte, Espirrolho plantou a semente no solo. Uma explosão de luz ocorreu, e a floresta começou a se regenerar. Árvores renasceram, flores desabrocharam, e os segredos da floresta se espalharam, conectando cada ser vivo com uma magia renovada.

Espirrolho voltou à vila, agora conhecido como o Protetor da Origem. Seu nome, antes motivo de risadas, tornou-se um símbolo de respeito e gratidão. Ele sabia que o trabalho não estava terminado, mas agora, com a floresta em equilíbrio, ele estava pronto para ouvir seus sussurros novamente.
"""

In [41]:
Documento = [Document(page_content=texto)]

In [42]:
# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(Documento)

In [43]:
# Criar embeddings e indexar no FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [44]:
system_template = """ Você é um amante de boas histórias e irá responder perguntas sobre elas. Diga apenas sobre as histórias e não invente nada

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [45]:
# Definição do modelo
llm = ChatOpenAI(model="gpt-4")

# Construir a pipeline RAG
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [47]:
question = "Quem é a guardiã?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: A Guardiã Verde é uma dríade sábia que vive no coração da floresta na história. Ela possui conhecimento profundo sobre as coisas da floresta e desempenha um papel crucial ao informar Espirrolho sobre a importância da Semente da Origem e o perigo que a floresta está enfrentando devido ao Feiticeiro Cinzento.
